In [1]:
!nvidia-smi

Wed Sep  3 06:46:01 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.247.01             Driver Version: 535.247.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        Off | 00000000:01:00.0  On |                  Off |
|  0%   45C    P8              24W / 450W |    555MiB / 24564MiB |      6%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from transformers import pipeline

In [3]:
model_id = "openai/gpt-oss-20b"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype="auto",
    device_map="auto",
)

Fetching 40 files:   0%|          | 0/40 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [4]:
!nvidia-smi

Wed Sep  3 06:46:13 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.247.01             Driver Version: 535.247.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        Off | 00000000:01:00.0  On |                  Off |
|  0%   47C    P2              68W / 450W |  14142MiB / 24564MiB |      7%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
print(pipe.model)

GptOssForCausalLM(
  (model): GptOssModel(
    (embed_tokens): Embedding(201088, 2880, padding_idx=199999)
    (layers): ModuleList(
      (0-23): 24 x GptOssDecoderLayer(
        (self_attn): GptOssAttention(
          (q_proj): Linear(in_features=2880, out_features=4096, bias=True)
          (k_proj): Linear(in_features=2880, out_features=512, bias=True)
          (v_proj): Linear(in_features=2880, out_features=512, bias=True)
          (o_proj): Linear(in_features=4096, out_features=2880, bias=True)
        )
        (mlp): GptOssMLP(
          (router): GptOssTopKRouter()
          (experts): Mxfp4GptOssExperts()
        )
        (input_layernorm): GptOssRMSNorm((2880,), eps=1e-05)
        (post_attention_layernorm): GptOssRMSNorm((2880,), eps=1e-05)
      )
    )
    (norm): GptOssRMSNorm((2880,), eps=1e-05)
    (rotary_emb): GptOssRotaryEmbedding()
  )
  (lm_head): Linear(in_features=2880, out_features=201088, bias=False)
)


In [6]:
num_parameters = 0
parameters_state_dict = pipe.model.state_dict()
for key, value in parameters_state_dict.items():
    print(key, value.size(), value.dtype, value.numel())
    num_parameters += value.numel()
print(f"Number of parameters: {num_parameters}")

model.embed_tokens.weight torch.Size([201088, 2880]) torch.bfloat16 579133440
model.layers.0.self_attn.sinks torch.Size([64]) torch.bfloat16 64
model.layers.0.self_attn.q_proj.weight torch.Size([4096, 2880]) torch.bfloat16 11796480
model.layers.0.self_attn.q_proj.bias torch.Size([4096]) torch.bfloat16 4096
model.layers.0.self_attn.k_proj.weight torch.Size([512, 2880]) torch.bfloat16 1474560
model.layers.0.self_attn.k_proj.bias torch.Size([512]) torch.bfloat16 512
model.layers.0.self_attn.v_proj.weight torch.Size([512, 2880]) torch.bfloat16 1474560
model.layers.0.self_attn.v_proj.bias torch.Size([512]) torch.bfloat16 512
model.layers.0.self_attn.o_proj.weight torch.Size([2880, 4096]) torch.bfloat16 11796480
model.layers.0.self_attn.o_proj.bias torch.Size([2880]) torch.bfloat16 2880
model.layers.0.mlp.router.weight torch.Size([32, 2880]) torch.bfloat16 92160
model.layers.0.mlp.router.bias torch.Size([32]) torch.bfloat16 32
model.layers.0.mlp.experts.gate_up_proj_bias torch.Size([32, 5760

In [9]:
messages = [
    {"role": "user", "content": "Explain quantum mechanics clearly and concisely."},
]

outputs = pipe(
    messages,
    max_new_tokens=1024,
)
print(outputs[0]["generated_text"][-1]["content"])

analysisWe need to explain quantum mechanics clearly and concisely. We can aim for a general overview: key principles: wave-particle duality, superposition, uncertainty principle, quantization, measurement, Schrödinger equation, entanglement. Keep concise but clear. Use analogies. Probably 300-500 words. Let's produce.assistantfinal## Quantum Mechanics in a Nutshell

| **What** | **How it works** | **Why it matters** |
|----------|------------------|--------------------|
| **Wave‑particle duality** | Light and matter behave like both particles and waves. A photon can travel as a stream of quanta *and* produce an interference pattern like waves. | Explains phenomena that classical physics cannot (e.g., diffraction of electrons). |
| **Quantization** | Certain properties (energy, angular momentum) can only take discrete values, not a continuous range. | Leads to the existence of atomic energy levels, the emission/absorption spectra of atoms. |
| **Superposition** | A quantum system can b

In [10]:
messages = [
    {
        "role": "user",
        "content": "how much memory to reserve for gpt oss on a 4090 rtx gpu",
    },
]

outputs = pipe(
    messages,
    max_new_tokens=1024,
)
print(outputs[0]["generated_text"][-1]["content"])

analysisWe need to interpret the question: "how much memory to reserve for gpt oss on a 4090 rtx gpu". GPT OSS likely refers to GPT-Chat open-source? Or GPT on GPU? Maybe the user wants to know how much GPU VRAM to reserve for GPT (like a large language model) on an RTX 4090. They might be asking about memory usage for the GPU when running GPT models (like GPT-4, GPT-3.5) on a 4090, and how much memory to set aside/reserve for the library. Or maybe "GPT OSS" refers to "gpt-oss" or "gpt-oss" open source version? Might be referencing "GPT-OSS" like GPT-3 open source? But likely the user wants to know how much memory they need to allocate for running a GPT model on an RTX 4090. The RTX 4090 has 24GB of GDDR6X memory. So how much memory to reserve? It depends on model size (e.g., GPT-2, GPT-3, GPT-4). For GPT-2 1.5B, you can run with ~4-5GB. GPT-3 175B would need huge memory, not possible. GPT-4? The largest open source GPT model? Might be GPT-NeoX 20B? That would need ~16GB? Let's think.


In [11]:
messages = [
    {
        "role": "user",
        "content": "Can you show me a Python code example for a simple web server?",
    },
]

outputs = pipe(
    messages,
    max_new_tokens=1024,
)
print(outputs[0]["generated_text"][-1]["content"])

analysisWe need to provide Python code example for a simple web server. The user didn't specify which framework or library. We can give a simple example using built-in http.server, or using Flask. Since they said "simple web server", maybe use the built-in http.server to keep example minimal. Provide explanation. Also mention how to run it. Provide code that listens on port 8080 and serves static files. Or a minimal Flask server that responds with "Hello, World!". It's up to us. Provide code for both? The question is open: "Can you show me a Python code example for a simple web server?" So providing a minimal example is fine. Let's show two options: built-in HTTPServer and Flask. Provide both. We'll keep code short. Also mention dependencies. Maybe ask if they want static files or dynamic. But we can provide a single example with built-in library. Let's include the simple HTTPServer example:

```
from http.server import BaseHTTPRequestHandler, HTTPServer

class SimpleHandler(BaseHTTPRe

In [12]:
messages = [
    {
        "role": "user",
        "content": "Write some pytorch code an intersection across two sparse tensors. Sparse tensors are defined as a tuple of 2 tensors : first tensor is lengths of values, the second tensor is coallesced tensor with all values. THe length of first tensor is the batch_size. DO NOT loop over the batch_size. do batched operations as vectorized directly in pytorch",
    },
]

outputs = pipe(
    messages,
    max_new_tokens=4096,
)
print(outputs[0]["generated_text"][-1]["content"])

analysisWe need to write PyTorch code for intersection across two sparse tensors. They are defined as tuple of two tensors: first tensor is lengths of values, second tensor is coalesced tensor with all values. The length of first tensor is batch_size.

We need to compute intersection across two sparse tensors. Means for each batch element, find common indices? We need to do batched operations, no looping over batch_size. Use vectorized operations.

We can think of each sparse representation: lengths: shape (batch_size), values: shape (total_nonzero,...) maybe with indices? Wait "coalesced tensor with all values" - maybe they mean the value tensor of the sparse matrix? Actually sparse tensors usually have indices and values. But here they say a tuple: (lengths, coalesced tensor with all values). So they likely store a list of tensors per batch in a flattened representation. For each batch i, length[i] is number of elements, and values is a contiguous tensor of shape (total_len,...). So 

In [13]:
import torch

messages = [
    {"role": "user", "content": "Explain how neural networks process text data."},
]


prof = torch.profiler.profile(
    activities=[
        torch.profiler.ProfilerActivity.CPU,
        torch.profiler.ProfilerActivity.CUDA,
    ],
    schedule=torch.profiler.schedule(wait=1, warmup=1, active=1, repeat=1),
    on_trace_ready=torch.profiler.tensorboard_trace_handler("./tensorboard"),
    # used when outputting for tensorboard
)
prof.start()

for i in range(5):
    # Run the model inference
    outputs = pipe(
        messages,
        max_new_tokens=64,  # Using smaller token count for profiling
    )
    prof.step()
prof.stop()
print("\n--- Profiler Results (Table) ---")
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=100))


--- Profiler Results (Table) ---
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         0.00%       0.000us         0.00%       0.000us       0.000us        2.154s       406.42%        2.154s        2.154s             1  
                                          ProfilerStep*        44.61%     961.194ms       100.00%        2.155s        2.155s       0.000us         0.00%     344.454ms     3